<a href="https://colab.research.google.com/github/sosodoit/yolov5/blob/main/YOLOv5_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<ul>Task 0 (YOLOv5 note)
<dl>용어, 개념, 궁금한 문제들 기록 (<a href="https://colab.research.google.com/drive/1u8S1vF5g9dSR8oDBJxjqiY9AH0C43jjF?usp=sharing">link</a>)</dl>
</ul>

<ol type="I">
  <li>Task 1 (사전작업)</li>
  <dl>YOLOv5 모델작업에 들어가기전, 데이터를 확인하고 정제한 작업 내용을 담고 있습니다(<a href="https://colab.research.google.com/drive/1fBpA9nP9l9pPbSCTFzNk1J2amQNDTZbQ?usp=sharing">link</a>)</dl>
  <ul type="square">
  <li>Putting data into a folder</li>
  <li>Convert the Annotations into the YOLOv5 Format</li>
  <li>Testing the annotations</li>
  <li>Partition the Dataset</li>
  </ul>
<strong>
  <li>Task 2 (학습)</li>
  <dl>YOLOv5 환경설정부터 학습한 작업내용을 담고 있습니다(here)</dl>
  <ul type="square">
  <li>Set up the environment</li>
  <li>Data Config File</li>
  <li>Hyper-parameter Config File</li>
  <li>Custom Network Architecture</li>
  <li>Train the Model</li>
  </ul>
</strong>
  <li>Task 3 (결과)</li>
  <dl>학습 후 테스트와 성능평가의 결과내용을 담고 있습니다(<a href="https://colab.research.google.com/drive/1---iUi0KXpVVjn95BNmsDyScnzLmFY6p?usp=sharing">link</a>)</dl>
  <ul type="square">
  <li>Test</li>
  <li>Computing the mAP on test dataset</li>
  </ul>
</ol>


---
---
---
# (실행1)Set up the environment
한 번 진행 후부터는 경로설정과 라이브러리 임포트만 해주면 된다(단, pip 업데이트는 해줘야함).
```python
########## 경로설정 ##########
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/객체탐지
!pip install --upgrade pip
########## 라이브러리 ##########
import os 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import shutil
import torch
from IPython.display import Image, clear_output
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
```


## Colab Setup

In [ ]:
########## 경로설정 ##########
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/객체탐지
!ls

In [ ]:
########## Clone YOLOv5 GitHub repository ##########
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
########## 주의: yolov5 경로에서 진행해야함 ##########
%cd yolov5 

In [ ]:
########## YOLOv5를 위한 패키지 설치 ##########
!pip install -r requirements.txt

In [ ]:
########## 왜인지는 모르나 pip 업데이트를 해줘야 학습진행시 에러가 안남 ##########
!pip install --upgrade pip

In [ ]:
import os 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import shutil

import torch
from IPython.display import Image, clear_output
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

**참고: Anaconda에서 진행할 시 YOLOv5 환경설정**

```python
##yolov5 아나콘다 가상환경 생성
conda create -n yolov5 python=3.8
conda activate yolov5
##yolov5 깃헙 클론
!git clone https://github.com/ultralytics/yolov5.git
##PyTorch ( [cuda 버전 확인](https://pytorch.org/get-started/previous-versions/) )
##yolov5는 최소 python>=3.8, pythorch>=1.7을 만족해야함
conda install pytorch==1.9.0 torchvision==0.8.1 torchaudio==0.7.2 cudatoolkit=11.0 -c pytorch 
pip install matplotlib tqdm opencv-python pillow PyYAML scipy tensorboard
conda install ipykernel
python -m ipykernel install --user --name 
yolov5 --display-name yolov5Kenrel
#혹은
pip install -r requirements.txt
pip install --upgrade pip
```

**참고: Colab 서버 스펙 확인**

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13786061021699090796, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11345264640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15430327371922178359
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [2]:
import platform
platform.platform()

'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic'

In [3]:
!cat /etc/issue.net

Ubuntu 18.04.5 LTS


In [4]:
!cat /proc/meminfo

MemTotal:       13302924 kB
MemFree:         9463688 kB
MemAvailable:   12121728 kB
Buffers:          128576 kB
Cached:          2571424 kB
SwapCached:            0 kB
Active:          1308036 kB
Inactive:        2171776 kB
Active(anon):     648436 kB
Inactive(anon):     2492 kB
Active(file):     659600 kB
Inactive(file):  2169284 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               164 kB
Writeback:             0 kB
AnonPages:        779828 kB
Mapped:           563808 kB
Shmem:              3232 kB
KReclaimable:     141264 kB
Slab:             196448 kB
SReclaimable:     141264 kB
SUnreclaim:        55184 kB
KernelStack:        4980 kB
PageTables:         8500 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    3532672 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       46696 kB
VmallocChunk:          0 kB
Percpu:          

In [5]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   38G   32G  55% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
tmpfs           6.4G   40K  6.4G   1% /var/colab
/dev/sda1        76G   42G   35G  55% /opt/bin/.nvidia
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [7]:
!nvidia-smi

Fri Sep  3 14:19:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    61W / 149W |    121MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
!python --version

Python 3.7.11


# 수정이 필요한 작업(완료)
- yolov5/data/cars.yaml
- yolov5/data/hyps/hyp.scratch.yaml (default)
- yolov5/model/yolov5s.yaml

## Data Config File
모델을 학습시키려는 데이터셋에 대한 세부 정보는 데이터 구성 YAML 파일에 의해 정의됨.
```python
# 경로: yolov5/data/cars.yaml
train: "/content/gdrive/MyDrive/객체탐지/dataset/train.txt"
val: "/content/gdrive/MyDrive/객체탐지/dataset/val.txt"

nc: 4
names: [Vehicle_Car, Vehicle_Bus, Vehicle_Motorcycle, Vehicle_Unknown]
```

In [ ]:
########## cars.yaml 파일 생성하는 작업##########
%cd /content/gdrive/My Drive/객체탐지/yolov5

########## 현재 cars.yaml파일 내용 확인##########
import yaml
with open('data/cars.yaml', 'r') as f:
  data = yaml.load(f)

print(data)

In [91]:
########## train-val에 있는 데이터 목록 가져오기 ##########
img_list = glob('../dataset/images/train/*')
val_img_list = glob('../dataset/images/val/*')

print(len(img_list))
print(len(val_img_list)) 

1854
464


In [92]:
########## 목록 리스트를 담은 텍스트 파일을 아래 경로에 생성 ##########
with open('../dataset/train.txt', 'w') as f:
  f.write('\n'.join(img_list)+'\n')

with open('../dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list)+'\n')

In [93]:
########## cars.yaml 파일에 train-val 경로를 위에서 생성한 텍스트파일 경로로 수정해주면 cars.yaml파일 작업완료 ##########
data['train'] = '/content/gdrive/MyDrive/객체탐지/dataset/train.txt'
data['val'] = '/content/gdrive/MyDrive/객체탐지/dataset/val.txt'

with open('/content/gdrive/MyDrive/객체탐지/yolov5/data/cars.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '/content/gdrive/MyDrive/객체탐지/dataset/train.txt', 'val': '/content/gdrive/MyDrive/객체탐지/dataset/val.txt', 'nc': 4, 'names': ['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown']}


## Hyper-parameter Config File
하이퍼파라미터 구성 파일은 신경망의 하이퍼파라미터를 정의하는 데 도움이 된다. 우리는 기본 데이터/hyp.scratch.yaml을 사용하고 있다.
```python
#경로: yolov5/data/hyps/hyp.scratch.yaml
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
```

## Custom Network Architecture
YOLOv5를 사용하면 미리 정의된 네트워크 중 사용자 지정 아키텍처 및 앵커를 정의할 수 있다. 이를 위해 사용자 정의 가중치 구성 파일을 정의해야 한다. 나는 가벼운 yolov5s.yaml을 사용했다. **주의: 클래스 갯수에 맞게 nc를 변경해줘야한다.**

```python
# 경로: yolov5/model/yolov5s.yaml
# Parameters
nc: 4  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
```

# (실행2)Train the Model
아직 데이터셋이 작고 이미지당 객체가 많지 않기 때문에 일을 단순하게 유지하고 과적합을 피하기 위해 가장 작은 pretrained모델 yolo5s부터 시작한다. 배치 크기는 32, 이미지 크기는 416을 유지하고 100 Epoch 동안 학습을 시켰다. 모델을 메모리에 맞추는 데 문제가 있는 경우:

- 배치사이즈를 줄이거나(32 -> 4)
- 더 작은 네트워크를 사용하거나
- 이미지 사이즈를 줄여보는

방법이 있다.

```python
!python train.py --img 416 --batch 32 --epochs 100 --data /content/gdrive/MyDrive/객체탐지/yolov5/data/cars.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5_result
#Logging results to runs/train/yolov5_result2
```

In [ ]:
!python train.py --img 416 --batch 32 --epochs 100 --data /content/gdrive/MyDrive/객체탐지/yolov5/data/cars.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5_result

train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/gdrive/MyDrive/객체탐지/yolov5/data/cars.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=yolov5_result, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=30
github: ⚠️ YOLOv5 is out of date by 2 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 2021-9-1 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.

In [ ]:
!python train.py --img 416 --batch 4 --epochs 100 --data /content/gdrive/MyDrive/객체탐지/yolov5/data/cars.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_results

In [ ]:
# 실행 x
# Start tensorboard 실시간으로 학습과정 살펴보기 위함, 참조할 폴더 runs
# validation result도 볼 수 있음. 복잡하다 싶으면 그냥 runs/train 폴더에서 확인하면됨.
%load_ext tensorboard
%tensorboard --logdir runs

# (실행3)Test

**참고**
- train.py: 데이터 로더는 속도와 정확도의 타협을 위해 설계되었다.
- val.py: 검증 데이터 세트에서 최고의 mAP를 얻도록 설계되었다.
- detect.py: 최고의 추론 결과를 위해 설계되었다.

```python
!python detect.py --img 416 --source /content/gdrive/MyDrive/객체탐지/dataset/images/val --weights /content/gdrive/MyDrive/객체탐지/yolov5/runs/train/yolov5_result2/weights/best.pt --conf-thres 0.4
```

In [ ]:
!python detect.py --img 416 --source /content/gdrive/MyDrive/객체탐지/dataset/images/val --weights /content/gdrive/MyDrive/객체탐지/yolov5/runs/train/yolov5_result2/weights/best.pt --conf-thres 0.4

# (실행4)Performance evaluation
YOLO에서는 모델의 성능(정확도)을 mAP를 통해 확인한다. mAP가 높을수록 정확하고, 작을수록 부정확하다.

* [colab - mAP 계산하러가기](https://drive.google.com/file/d/1Xh_BWS90mx13MQIfmqLa9ijjxpaUO3lS/view?usp=sharing)

---
# **Issue**

- Colab gpu 할당 시간 넘어서 멈춘다...
- <del>바운딩박스가 객체 인식하는게 이상함</del>(Task1에서 해결완료)
- FPS는 어떻게 확인할 수 있을까?
- 차종분류모델, xml작업?
- 번호판 Detection 먼저?

# Questions
- 사진 크기가 1280x672, 1920x1080 인데 학습 사진 크기를 416으로함. 영향없나?
- train, val 나누고 train.py 돌리는데 val데이터로 학습된 걸 평가중인건가? val.py는 왜있는건가!! 더 최상의 성능평가가 나올 수 있도록 짜여진 코드?
- coco 에 car detection이 성능좋게 만들어져있는데 이것을 이용하고 + 우리 데이터를 같이 사용중인건가? 전이학습이라?